In [1]:
!mkdir data
import urllib.request    
urllib.request.urlretrieve("https://raw.githubusercontent.com/UniversalDependencies/UD_Turkish-IMST/master/tr_imst-ud-train.conllu", "data/"+"train.conllu")
urllib.request.urlretrieve("https://raw.githubusercontent.com/UniversalDependencies/UD_Turkish-IMST/master/tr_imst-ud-dev.conllu", "data/"+"dev.conllu")
urllib.request.urlretrieve("https://raw.githubusercontent.com/UniversalDependencies/UD_Turkish-IMST/master/tr_imst-ud-test.conllu", "data/"+"test.conllu")

urllib.request.urlretrieve("https://huggingface.co/dbmdz/bert-base-turkish-128k-cased/raw/main/vocab.txt", "data/"+"vocab.txt")

('data/vocab.txt', <http.client.HTTPMessage at 0x7f4e984870d0>)

In [2]:
!pip install conllu
!pip install spacy
#!spacy convert train.conllu ./ -l tr
from io import open
from conllu import parse_incr

def to_txt(file_name,labels):
  json_file = []

  data_file = open("data/"+file_name+".conllu", "r", encoding="utf-8")
  for tokenlist in parse_incr(data_file):
      json_file.append([i for i in tokenlist])
    
  file_w = open("data/"+file_name+'.txt', 'w')
  
  for sentence in json_file:
    for word in sentence:
      labels.add(word.get("upos",""))
      file_w.write(word.get("form","")+" ")
      file_w.write(word.get("upos","")+"\n")
    file_w.write("\n")
  return labels


labels = set()
labels = to_txt("train",labels)
labels = to_txt("test",labels)
labels = to_txt("dev",labels)

In [4]:
file_l = open("labels.txt", 'w')
for sentence in list(labels):
  file_l.write(sentence+"\n")

In [5]:
!wget https://raw.githubusercontent.com/huggingface/transformers/v3.0.2-docs/examples/token-classification/run_ner.py
!wget https://raw.githubusercontent.com/huggingface/transformers/v3.0.2-docs/examples/token-classification/utils_ner.py
#!wget https://raw.githubusercontent.com/huggingface/transformers/v3.0.2-docs/examples/token-classification/tasks.py

--2021-02-27 15:03:18--  https://raw.githubusercontent.com/huggingface/transformers/v3.0.2-docs/examples/token-classification/run_ner.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11184 (11K) [text/plain]
Saving to: ‘run_ner.py’

run_ner.py          100%[===================>]  10.92K  --.-KB/s    in 0s      

2021-02-27 15:03:18 (97.5 MB/s) - ‘run_ner.py’ saved [11184/11184]

--2021-02-27 15:03:18--  https://raw.githubusercontent.com/huggingface/transformers/v3.0.2-docs/examples/token-classification/utils_ner.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, a

In [6]:
OUTPUT_DIR = "bert-pos" 

!git clone https://github.com/huggingface/transformers
%cd transformers
!git checkout v3.0.2-docs 
!pip install .
!pip install -r ./examples/requirements.txt
%cd ..

Cloning into 'transformers'...
remote: Enumerating objects: 64769, done.
remote: Total 64769 (delta 0), reused 0 (delta 0), pack-reused 64769
Receiving objects: 100% (64769/64769), 48.68 MiB | 29.60 MiB/s, done.
Resolving deltas: 100% (45917/45917), done.
/content/transformers
Branch 'v3.0.2-docs' set up to track remote branch 'v3.0.2-docs' from 'origin'.
Switched to a new branch 'v3.0.2-docs'
Processing /content/transformers
     |████████████████████████████████| 3.0MB 17.2MB/s 
     |████████████████████████████████| 1.2MB 49.9MB/s 
     |████████████████████████████████| 890kB 39.1MB/s 
  Created wheel for transformers: filename=transformers-3.0.2-cp37-none-any.whl size=757879 sha256=e9671c9d32878fcf6e50a278d07aad6274dcf8972e82fcf366c181af590e9b20
  Stored in directory: /tmp/pip-ephem-wheel-cache-9ehjizvy/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=2b41207715086b32

/content


In [2]:
OUTPUT_DIR = "bert-pos"
!python run_ner.py --data_dir ./data \
--labels ./labels.txt \
--model_name_or_path dbmdz/bert-base-turkish-128k-cased  \
--output_dir $OUTPUT_DIR \
--max_seq_length 128 \
--num_train_epochs 10 \
--per_device_train_batch_size 16 \
--save_steps 10000 \
--seed 41 \
--do_train \
--do_eval \
--do_predict

2021-02-27 15:10:55.419800: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/27/2021 15:10:56 - INFO - transformers.training_args -   PyTorch: setting up devices
02/27/2021 15:10:56 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
02/27/2021 15:10:56 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='bert-ner', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Feb27_15-10-56_45b14d1e625f', logging_first_step=False, logging_steps=500, save_steps=10000, save_tot

In [52]:
%cd content

/content


In [54]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel

tokenizer = AutoTokenizer.from_pretrained("./bert-ner")
model = AutoModel.from_pretrained("./bert-ner")

In [6]:
test_sentence = """Mustafa Kemal Ankara'ya gidiyor."""
ner = pipeline('ner', model=model, tokenizer=tokenizer)
res = ner(test_sentence)

In [7]:
for r in res:
    print(r)

{'word': '[CLS]', 'score': 0.9993565678596497, 'entity': 'PUNCT', 'index': 0}
{'word': 'Mustafa', 'score': 0.9997118711471558, 'entity': 'PROPN', 'index': 1}
{'word': 'Kemal', 'score': 0.9997426271438599, 'entity': 'PROPN', 'index': 2}
{'word': 'Ankara', 'score': 0.9997888803482056, 'entity': 'PROPN', 'index': 3}
{'word': "'", 'score': 0.8607736229896545, 'entity': 'ADP', 'index': 4}
{'word': 'ya', 'score': 0.41962313652038574, 'entity': 'CCONJ', 'index': 5}
{'word': 'gidiyor', 'score': 0.9998812675476074, 'entity': 'VERB', 'index': 6}
{'word': '.', 'score': 0.9998824000358582, 'entity': 'PUNCT', 'index': 7}
{'word': '[SEP]', 'score': 0.9998824000358582, 'entity': 'PUNCT', 'index': 8}
